In [1]:
import ffn
import datetime

In [2]:
tickers = ['AAPL','MSFT','NVDA','GOOGL','TSLA','MSTR','PLTR','IAU','SPLG','AMZN', 'META', 'SCHG']

In [3]:
returns = ffn.get(tickers, start='2024-11-12').to_returns().dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
returns.calc_mean_var_weights().as_format('.2%')

aapl     42.26%
msft     10.92%
nvda      0.00%
googl     0.00%
tsla     15.74%
mstr      1.17%
pltr     11.94%
iau       8.74%
splg      0.00%
amzn      5.73%
meta      3.50%
schg      0.00%
dtype: object

# Recent Stock Performance

In [17]:
start_date = datetime.date.today() - datetime.timedelta(days=3)

In [18]:
start_date

datetime.date(2024, 12, 9)

In [19]:
prices = ffn.get(tickers, start=start_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
final = prices.pct_change().dropna().iloc[-1:].T.reset_index()

In [21]:
final.columns = ['ticker', 'pct']

In [22]:
final

,ticker,pct
0,aapl,-0.005166
1,msft,0.012767
2,nvda,0.031391
3,googl,0.055247
4,tsla,0.059303
5,mstr,0.090321
6,pltr,0.022852
7,iau,0.009246
8,splg,0.007898
9,amzn,0.023196


In [23]:
best = final.nlargest(5, 'pct')

In [24]:
best

,ticker,pct
5,mstr,0.090321
4,tsla,0.059303
3,googl,0.055247
2,nvda,0.031391
9,amzn,0.023196


In [25]:
worst = final.nsmallest(5, 'pct')

In [26]:
worst

,ticker,pct
0,aapl,-0.005166
8,splg,0.007898
7,iau,0.009246
1,msft,0.012767
11,schg,0.016973


## Cumulative Returns Analysis

This calculation is helpful for analyzing the overall performance of different assets within a portfolio and can also assist in comparing individual assets to an index like the S&P 500.

In [27]:
cumulative_returns = prices.calc_total_return()

In [28]:
print(cumulative_returns)

aapl    -0.001054
msft     0.006659
nvda     0.003602
googl    0.114216
tsla     0.089741
mstr     0.126074
pltr     0.000690
iau      0.022523
splg     0.004639
amzn     0.018444
meta     0.031146
schg     0.014119
dtype: float64


## Sharpe Ratio Calculation

The Sharpe Ratio is a popular metric that allows investors to assess the risk-adjusted return of an investment. A higher Sharpe Ratio indicates that the asset is providing a higher return for a given level of risk. The ffn library allows you to calculate the Sharpe Ratio easily

In [29]:
sharpe_ratios = prices.calc_stats()

In [30]:
sharpe_ratios.display()

Stat                 aapl        msft        nvda        googl            tsla           mstr              pltr        iau         splg        amzn        meta        schg
-------------------  ----------  ----------  ----------  ---------------  -------------  ----------------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2024-12-09  2024-12-09  2024-12-09  2024-12-09       2024-12-09     2024-12-09        2024-12-09  2024-12-09  2024-12-09  2024-12-09  2024-12-09  2024-12-09
End                  2024-12-11  2024-12-11  2024-12-11  2024-12-11       2024-12-11     2024-12-11        2024-12-11  2024-12-11  2024-12-11  2024-12-11  2024-12-11  2024-12-11
Risk-free rate       0.00%       0.00%       0.00%       0.00%            0.00%          0.00%             0.00%       0.00%       0.00%       0.00%       0.00%       0.00%

Total Return         -0.11%      0.67%       0.36%       11.42%           8.97%          12.61%            0.07%       2

## Rolling Volatility

In [31]:
rolling_vol = prices.rolling(3).std()

In [32]:
print(rolling_vol.tail())

               aapl      msft      nvda      googl      tsla       mstr  \
Date                                                                      
2024-12-09      NaN       NaN       NaN        NaN       NaN        NaN   
2024-12-10      NaN       NaN       NaN        NaN       NaN        NaN   
2024-12-11  0.67656  2.831156  2.317148  10.015769  17.86303  23.897313   

                pltr       iau      splg      amzn      meta      schg  
Date                                                                    
2024-12-09       NaN       NaN       NaN       NaN       NaN       NaN  
2024-12-10       NaN       NaN       NaN       NaN       NaN       NaN  
2024-12-11  0.921214  0.567657  0.281483  2.761033  9.804278  0.258322  


## Portfolio Optimization

This method of portfolio optimization leverages historical data to create an allocation that minimizes risk while targeting a desired return.

In [33]:
portfolio = prices.calc_stats().prices

In [34]:
portfolio

,aapl,msft,nvda,googl,tsla,mstr,pltr,iau,splg,amzn,meta,schg
Date,,,,,,,,,,,,
2024-12-09,246.750000,446.019989,138.809998,175.369995,389.790009,365.339996,72.459999,50.169998,71.129997,226.089996,613.570007,28.477911
2024-12-10,247.770004,443.329987,135.070007,185.169998,400.989990,377.320007,70.889999,50.830002,70.900002,225.039993,619.320007,28.398001
2024-12-11,246.490005,448.989990,139.309998,195.399994,424.769989,411.399994,72.510002,51.299999,71.459999,230.259995,632.679993,28.879999


In [35]:
weights = portfolio.calc_mean_var_weights().as_format('.2%')

In [36]:
print(weights)

aapl     33.70%
msft     33.54%
nvda      4.30%
googl     0.00%
tsla      0.00%
mstr      0.00%
pltr      4.21%
iau       2.97%
splg      6.52%
amzn      6.31%
meta      6.98%
schg      1.47%
dtype: object


## Max Drawdown Analysis

The maximum drawdown is an important metric to evaluate the worst peak-to-trough decline during a specific period. It provides insight into how much value an investor could lose most during a downturn:

In [37]:
max_drawdown = prices.calc_max_drawdown().as_format('.2%')

In [38]:
print(max_drawdown)

aapl     -0.52%
msft     -0.60%
nvda     -2.69%
googl     0.00%
tsla      0.00%
mstr      0.00%
pltr     -2.17%
iau       0.00%
splg     -0.32%
amzn     -0.46%
meta      0.00%
schg     -0.28%
dtype: object


Consultas: https://chatgpt.com/c/675989fe-f91c-800b-a51c-36da7260404f